In [1]:
from datasets import load_dataset

passages = load_dataset("UKPLab/dapr", "MSMARCO-corpus", split="test")
queries = load_dataset("UKPLab/dapr", "MSMARCO-queries", split="test")
qrels_rows = load_dataset("UKPLab/dapr", "MSMARCO-qrels", split="test")

role_answer = load_dataset("elricwan/MSMARCO-queries-roleAns", split="train")

/root/miniconda3/envs/agent/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import sqlite3

# Create or open a database
conn = sqlite3.connect("passages.db")
cursor = conn.cursor()

# Create a table
# cursor.execute('''
#     CREATE TABLE IF NOT EXISTS passages (
#         _id TEXT PRIMARY KEY,
#         text TEXT,
#         title TEXT,
#         doc_id TEXT,
#         paragraph_no INTEGER,
#         total_paragraphs INTEGER,
#         is_candidate BOOLEAN
#     )
# ''')
# # Insert data into the table
# # Assuming `passages` is your dataset object
# for passage in passages:
#     cursor.execute('''
#         INSERT OR IGNORE INTO passages VALUES (?, ?, ?, ?, ?, ?, ?)
#     ''', (passage['_id'], passage['text'], passage['title'], passage['doc_id'], passage['paragraph_no'], passage['total_paragraphs'], passage['is_candidate']))

# conn.commit()

In [3]:
# Query by _id
def get_text_by_id(_id):
    cursor.execute('SELECT text FROM passages WHERE _id = ?', (_id,))
    result = cursor.fetchone()
    return result[0] if result else None

# Example usage
text = get_text_by_id('D2147834-0')

In [5]:
# convert role_answer to a searchable dictionary
role_answer_dict = {}
for item in role_answer:
    _id = item['_id']
    role_answer_dict[_id] = {}
    role_answer_dict[_id]['question'] = item['text']
    role_answer_dict[_id]['answers'] = item['answers']

In [6]:
from typing import Dict

qrels: Dict[str, Dict[str, float]] = {}
for qrel_row in qrels_rows:
    qid = qrel_row["query_id"]
    pid = qrel_row["corpus_id"]
    rel = qrel_row["score"]
    qrels.setdefault(qid, {})
    qrels[qid][pid] = rel

In [7]:
import ollama
import numpy as np
# Function to calculate embedding for each point
def calculate_embedding(point):
    embedding = ollama.embeddings(model='mxbai-embed-large', prompt=point)["embedding"]
    return embedding

In [8]:
from src.milvus_database import connect_to_milvus, create_index, search_document_by_embedding

document_collection_name = "graphrag_document_collection"
connect_to_milvus()
#create_index(document_collection_name, "text_embedding")

Connected to Milvus


In [11]:
from pymilvus import Collection
collection = Collection(document_collection_name)
collection.num_entities

6010953

In [75]:
import json

with open('jsonl/data.json', 'r') as file:
    data = json.load(file)
with open('jsonl/retrieve.json', 'r') as file:
    retrieve = json.load(file)


In [106]:
# Function to compare the datasets without considering the prefix
def compare_retrieve(data1, data2):
    results = {}
    for key in data2:
        # sort data1 by distance
        sample_question = sorted(data1[key]['question'], key=lambda x: x[1])
        sample_answer = sorted(data1[key]['General'], key=lambda x: x[1])
       

        # Get the set of document IDs in data1 for the current category
        data1_question_ids = {item[0] for item in sample_question}
        data1_general_ids = {item[0] for item in sample_answer}
        
        # Get the set of document IDs in data2 for the current category
        data2_ids = set(data2[key].keys())
        
        # Check if all IDs from data2 exist in data1
        question_contains_all = data2_ids.issubset(data1_question_ids)
        answer_contains_all = data2_ids.issubset(data1_general_ids)

        # Check if there is any overlap between data1 and data2
        overlap_question = bool(data1_question_ids.intersection(data2_ids))
        overlap_answer = bool(data1_general_ids.intersection(data2_ids))
        
        # Store results
        results[key] = {
            'question_contains_all': question_contains_all,
            'answer_contains_all': answer_contains_all,
            'overlap_question': overlap_question,
            'overlap_answer': overlap_answer,
        }
       
    return results

In [108]:
print(len([key for key in results_question if results_question[key]['overlap_question']]))
print(len([key for key in results_question if results_question[key]['overlap_answer']]))

2302
2149


In [107]:
results_question = compare_retrieve(retrieve, qrels)
print(len([key for key in results_question if results_question[key]['question_contains_all']]))
print(len([key for key in results_question if results_question[key]['answer_contains_all']]))

2273
2119


In [111]:
aaa = [key for key in results_question if results_question[key]['question_contains_all']] + [key for key in results_question if results_question[key]['answer_contains_all']]
len(set(aaa))

2386

In [112]:
bbb = [key for key in results_question if results_question[key]['overlap_question']] + [key for key in results_question if results_question[key]['overlap_answer']]
len(set(bbb))

2412